In [1]:
!pip install transformers==4.52.4 pyngrok fastapi uvicorn python-multipart openai-whisper pypdf ffmpeg-python --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 15.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 97.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 

In [2]:

import os, torch, threading, time, socket, tempfile, logging, re
from pyngrok import ngrok, conf
from fastapi import FastAPI, UploadFile, File, Form, HTTPException, Depends
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from transformers import AutoTokenizer, AutoModelForCausalLM
import whisper
from pypdf import PdfReader
from pathlib import Path
from typing import Optional

In [ ]:
# CONFIG
# ========================
NGROK_TOKEN = "your_ngrok_token_here"
API_KEY = "123456"
TEMP_DIR = tempfile.mkdtemp()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [4]:
# ========================
# MODELS
# ========================
class Models:
    mistral_model = None
    mistral_tokenizer = None
    whisper_model = None

    @classmethod
    def load(cls):
        logger.info("🔄 Loading Kaggle models...")

        # Load Mistral
        logger.info("Loading Mistral...")
        cls.mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-Nemo-Instruct-2407")
        cls.mistral_model = AutoModelForCausalLM.from_pretrained(
            "mistralai/Mistral-Nemo-Instruct-2407",
            torch_dtype=torch.float16,
            device_map="auto",
            low_cpu_mem_usage=True
        ).eval()
        logger.info("✅ Mistral loaded")

        # Load Whisper (tiny to avoid GPU crash)
        logger.info("Loading Whisper...")
        cls.whisper_model = whisper.load_model("tiny")
        logger.info("✅ Whisper loaded")

        logger.info("✅ All Kaggle models ready!")

In [5]:
# ========================
# SECURITY
# ========================
security = HTTPBearer()

def verify(creds: HTTPAuthorizationCredentials = Depends(security)):
    if creds.credentials != API_KEY:
        raise HTTPException(401, "Unauthorized")
    return creds.credentials


In [6]:
# ========================
# OUTPUT PARSER
# ========================
def parse_model_output(raw_output: str, prompt: str, analysis_type: str) -> str:
    """
    Parse and clean the model output to remove prompt repetition and extract only the response.
    """
    # Remove the prompt from the output if it appears at the beginning
    output = raw_output
    
    # Try to find where the actual response starts
    # Common markers that indicate the start of the actual response
    markers = {
        "brief": ["Provide a brief summary", "Brief summary:", "Summary:"],
        "detailed": ["Detailed Summary:", "Detailed Analysis:", "Analysis:"],
        "main_topics": ["Main Topics:", "Topics:"],
        "keywords_focus": ["Keyword-Focused Analysis:", "Analysis:"],
        "bullet_points": ["Key Points:", "•", "-"],
        "question_answer": ["Q&A Summary:", "Q:", "Question:"]
    }
    
    # Find the last occurrence of the marker for this analysis type
    relevant_markers = markers.get(analysis_type, ["Analysis:", "Summary:"])
       
    # Try each marker
    for marker in relevant_markers:
        if marker in output:
            # Split at the last occurrence of the marker
            parts = output.split(marker)
            if len(parts) > 1:
                # Take everything after the last marker
                output = parts[-1].strip()
                break
    
    # Remove the prompt text if it's still there
    prompt_lines = prompt.split('\n')
    for line in prompt_lines:
        if len(line.strip()) > 20:  # Only remove substantial lines
            output = output.replace(line, '')
    
    # Remove common instruction patterns using regex
    patterns_to_remove = [
        r'User Keywords:.*?TEXT:',
        r'Provide a brief summary.*?(?=\n\n|\n[A-Z])',
        r'Provide a detailed summary.*?(?=\n\n|\n[A-Z])',
        r'Identify and explain.*?(?=\n\n|\n[A-Z])',
        r'Create a Q&A summary.*?(?=\n\n|\n[A-Z])',
        r'Analyze the text focusing.*?(?=\n\n|\n[A-Z])',
        r'Extract and explain.*?(?=\n\n|\n[A-Z])',
        r'For each topic:.*?(?=\n\n|\n[A-Z])',
        r'Each bullet should be.*?(?=\n\n|\n[A-Z])',
        r'\*\s*Topic name.*?(?=\n)',
        r'\*\s*Brief explanation.*?(?=\n)',
        r'\*\s*Key points.*?(?=\n)',
    ]
    
    for pattern in patterns_to_remove:
        output = re.sub(pattern, '', output, flags=re.DOTALL | re.IGNORECASE)
    
    # Clean up extra whitespace and newlines
    output = re.sub(r'\n{3,}', '\n\n', output)
    output = output.strip()
    
    # If output is too short or empty, return a fallback message
    if len(output) < 50:
        return "Analysis generated. Please note: The model output was too brief or could not be properly parsed."
    
    return output
 

In [8]:
# ========================
# HELPERS
# ========================
async def save_file(file: UploadFile, prefix: str) -> str:
    path = os.path.join(TEMP_DIR, f"{prefix}{Path(file.filename).suffix}")
    with open(path, 'wb') as f:
        f.write(await file.read())
    return path

def extract_pdf(path: str) -> str:
    return "\n".join([p.extract_text() or "" for p in PdfReader(path).pages])

def transcribe(path: str) -> str:
    return Models.whisper_model.transcribe(path)["text"]

def extract_audio_from_video(video_path: str, audio_path: str):
    import subprocess
    subprocess.run([
        'ffmpeg', '-i', video_path, '-vn', '-acodec',
        'libmp3lame', '-q:a', '2', audio_path, '-y'
    ], check=True, capture_output=True)

def generate_analysis(prompt: str, max_len: int = 400) -> str:
    inputs = Models.mistral_tokenizer(
        prompt, return_tensors="pt",
        truncation=True, max_length=2048
    ).to(Models.mistral_model.device)

    with torch.no_grad():
        outputs = Models.mistral_model.generate(
            **inputs,
            max_new_tokens=max_len,
            pad_token_id=Models.mistral_tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )

    return Models.mistral_tokenizer.decode(outputs[0], skip_special_tokens=True)

def build_analysis_prompt(text: str, analysis_type: str, keywords: str = "") -> tuple:
    """Build analysis prompt based on type and return (prompt, max_tokens)"""
    
    text_preview = text[:3000]  # Limit text for context
    keyword_text = f"Focus Keywords: {keywords}" if keywords else "No specific keywords provided"
    
    prompts = {
        "brief": (f"""You are a summarization expert. Analyze this text and provide ONLY a brief 2-3 sentence summary.

{keyword_text}

TEXT TO ANALYZE:
{text_preview}

Write your brief summary below (2-3 sentences only):
""", 200),
        
        "detailed": (f"""You are an analysis expert. Provide a comprehensive analysis of this text.

{keyword_text}

TEXT TO ANALYZE:
{text_preview}

Write a detailed analysis covering main themes, key points, evidence, and conclusions:
""", 600),
        
        "main_topics": (f"""You are a content analyzer. Extract the main topics from this text.

{keyword_text}

TEXT TO ANALYZE:
{text_preview}

List and explain each main topic with its key points:
""", 500),
        
        "keywords_focus": (f"""You are an expert analyzer. Focus your analysis on these specific keywords: {keywords}

TEXT TO ANALYZE:
{text_preview}

Analyze how these keywords appear in the text, their context, and significance:
""", 500),
        
        "bullet_points": (f"""You are a summarization expert. Extract the key points as clear bullet points.

{keyword_text}

TEXT TO ANALYZE:
{text_preview}

List the main points as bullet points:
""", 400),
        
        "question_answer": (f"""You are an expert analyzer. Create a Q&A format summary.

{keyword_text}

TEXT TO ANALYZE:
{text_preview}

Create questions and answers about this text:
""", 500)
    }
    
    return prompts.get(analysis_type, prompts["detailed"])


In [9]:
# ========================
# FASTAPI APP
# ========================
app = FastAPI(title="Kaggle Enhanced Analysis API", version="2.0")

@app.on_event("startup")
async def startup():
    Models.load()

@app.get("/")
async def root():
    return {"status":"online", "version": "2.0", "features": ["extraction", "multi-mode analysis", "output parser"]}

@app.get("/health")
async def health():
    return {
        "status": "healthy",
        "models": {
            "mistral": Models.mistral_model is not None,
            "whisper": Models.whisper_model is not None
        },
        "analysis_modes": ["brief", "detailed", "main_topics", "keywords_focus", "bullet_points", "question_answer"]
    }

@app.post("/extract")
async def extract(
    input_type: str = Form(...),
    file: UploadFile = File(...),
    token: str = Depends(verify)
):
    file_path = audio_path = None

    try:
        if input_type == "pdf":
            file_path = await save_file(file, "doc")
            text = extract_pdf(file_path)

        elif input_type == "audio":
            file_path = await save_file(file, "audio")
            text = transcribe(file_path)

        elif input_type == "video":
            file_path = await save_file(file, "video")
            audio_path = os.path.join(TEMP_DIR, "audio.mp3")
            extract_audio_from_video(file_path, audio_path)
            text = transcribe(audio_path)

        else:
            raise HTTPException(400, "Invalid input_type")

        return {"status":"success", "text":text, "length": len(text)}

    finally:
        for p in [file_path, audio_path]:
            if p and os.path.exists(p):
                os.remove(p)
@app.post("/analyze")
async def analyze(
    text: str = Form(...),
    analysis_type: str = Form("detailed"),
    keywords: str = Form(""),
    token: str = Depends(verify)
):
    """Enhanced analysis with multiple modes and output parsing"""
    try:
        logger.info(f"Analysis request - Type: {analysis_type}, Keywords: {keywords}")
        
        # Validate analysis type
        valid_types = ["brief", "detailed", "main_topics", "keywords_focus", "bullet_points", "question_answer"]
        if analysis_type not in valid_types:
            raise HTTPException(400, f"Invalid analysis_type. Must be one of: {valid_types}")
        
        # Build prompt based on analysis type
        prompt, max_tokens = build_analysis_prompt(text, analysis_type, keywords)
        
        # Generate analysis
        raw_output = generate_analysis(prompt, max_tokens)
        logger.info(f"Raw output length: {len(raw_output)}")
        
        # Parse and clean the output
        cleaned_output = parse_model_output(raw_output, prompt, analysis_type)
        logger.info(f"Cleaned output length: {len(cleaned_output)}")
        
        return {
            "status": "success",
            "analysis": cleaned_output,
            "analysis_type": analysis_type,
            "keywords": keywords,
            "text_length": len(text)
        }
    
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"Analysis error: {e}")
        raise HTTPException(500, f"Analysis failed: {str(e)}")


/tmp/ipykernel_89/2503106960.py:6: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [10]:
# ========================
# START SERVER WITH NGROK
# ========================
def get_port():
    with socket.socket() as s:
        s.bind(('', 0))
        return s.getsockname()[1]

port = get_port()
conf.get_default().auth_token = NGROK_TOKEN
url = ngrok.connect(port).public_url

print("\n" + "="*70)
print("🎉 KAGGLE ENHANCED ANALYSIS SERVER IS READY!")
print("="*70)
print(f"🔗 KAGGLE URL: {url}")
print(f"🔑 API KEY: {API_KEY}")
print("✨ NEW: Advanced output parser to remove prompt repetition")
print("="*70)
print("\n📋 ANALYSIS MODES:")
print("  • brief - Quick 2-3 sentence summary")
print("  • detailed - Comprehensive analysis")
print("  • main_topics - Extract and explain main topics")
print("  • keywords_focus - Focus on user keywords")
print("  • bullet_points - Key points in bullets")
print("  • question_answer - Q&A format summary")
print("="*70 + "\n")

threading.Thread(
    target=lambda: __import__('uvicorn').run(app, host="0.0.0.0", port=port),
    daemon=True
).start()

while True:
    time.sleep(1)

INFO:pyngrok.ngrok:Opening tunnel named: http-53347-06de9f13-a932-4bea-a95b-176a0fd57c55


INFO:pyngrok.process:Overriding default auth token
INFO:pyngrok.process.ngrok:t=2025-11-28T12:32:07+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2025-11-28T12:32:07+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
INFO:pyngrok.process.ngrok:t=2025-11-28T12:32:07+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
INFO:pyngrok.process.ngrok:t=2025-11-28T12:32:07+0000 lvl=info msg="FIPS 140 mode" enabled=false
INFO:pyngrok.process.ngrok:t=2025-11-28T12:32:07+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
INFO:pyngrok.process.ngrok:t=2025-11-28T12:32:07+0000 lvl=info msg="client session established" obj=tunnels.session
INFO:pyngrok.process.ngrok:t=2025-11-28T12:32:07+0000 lvl=info msg="tunnel session started" obj=tunnels.session
INFO:pyngrok.process.ngrok:t=2025-11-28T12:32:07+0000 lvl=info msg=start pg=/api/tunnels id=8bcb6ced032cc696
INFO:


🎉 KAGGLE ENHANCED ANALYSIS SERVER IS READY!
🔗 KAGGLE URL: https://isaac-thermolabile-deonna.ngrok-free.dev
🔑 API KEY: 123456
✨ NEW: Advanced output parser to remove prompt repetition

📋 ANALYSIS MODES:
  • brief - Quick 2-3 sentence summary
  • detailed - Comprehensive analysis
  • main_topics - Extract and explain main topics
  • keywords_focus - Focus on user keywords
  • bullet_points - Key points in bullets
  • question_answer - Q&A format summary



INFO:     Started server process [89]
INFO:     Waiting for application startup.
INFO:__main__:🔄 Loading Kaggle models...
INFO:__main__:Loading Mistral...


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

INFO:     156.203.167.177:0 - "GET /health HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2025-11-28T12:41:21+0000 lvl=info msg="join connections" obj=join id=4f19c6c85fc9 l=127.0.0.1:53347 r=156.203.167.177:37342


INFO:     156.203.167.177:0 - "GET /health HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2025-11-28T12:41:30+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


KeyboardInterrupt: 